## Import the Libraries

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
import numpy as np

## Load the dataset

In [ ]:
df = pd.read_csv("F:\Datasets\Crop_recommendation.csv", index_col=False)
df.drop("rainfall", axis=1, inplace=True)

## Encoding the Labels

In [ ]:
dataset = df.values
X = dataset[:, 0:6].astype(float)
Y = dataset[:, 6] 

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = to_categorical(encoded_Y)

## Neural Network with Softmax Activation

In [ ]:
def classification_model():
	model = Sequential()
	model.add(Dense(44, input_dim=6, activation='relu'))
	model.add(Dense(22, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

## Evaluate the model

In [ ]:
estimator = KerasClassifier(build_fn=classification_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Results accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

## Training the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.2, random_state=42)

model = classification_model()
model.fit(X_train, y_train, epochs=400, batch_size=5, verbose=0)
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)
print("Loss: ", loss)

## Predicting

In [ ]:
example = np.array([[90, 42, 43, 20.879744, 82.002744, 6.502985]])
predictions = model.predict(example)
class_labels = ['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas', 'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate', 'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee']

encoder.fit(class_labels)
predicted_labels_encoded = np.argmax(predictions, axis=1)
predicted_labels = encoder.inverse_transform(predicted_labels_encoded)

print("Predicted label:", predicted_labels)